In [2]:
import pandas as pd 
df = pd.read_csv("Churn_Modelling.csv")

In [3]:
import tensorflow
from tensorflow import keras
from keras import Sequential
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [6]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [7]:
df.duplicated().value_counts()

False    10000
dtype: int64

In [8]:
df["Exited"].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

I am choosing to undersample the majority class, because the minority class is the most important one to predict, and the real data should be kept.

In [9]:
df = df.drop(["CustomerId", "RowNumber", "Surname" ], axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [10]:
df["Geography"].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df.select_dtypes("object"):
    df[col] = le.fit_transform(df[col])

df
    

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,1,1,42,3,75075.31,2,1,0,92888.52,1


I will do it after split
1. split
2. scaling
3. undersampling

In [12]:
Y = df["Exited"]
X = df.drop(["Exited"], axis=1)

In [13]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,Y, test_size=0.25, random_state=42, shuffle=True)

In [14]:
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [15]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
XtrainR, YtrainR = rus.fit_resample(Xtrain, ytrain)


In [16]:
from sklearn.ensemble import RandomForestClassifier
params = {
    "n_estimators" : [170,200,230],
    "max_depth": [None, 3,5,7]
}

from sklearn.model_selection import GridSearchCV
#wont use cv - cross validation since it is already good with overfitting
grid = GridSearchCV(RandomForestClassifier(), param_grid= params, scoring="recall", verbose=2)

grid.fit(XtrainR, YtrainR)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[CV] END ...................max_depth=None, n_estimators=170; total time=   0.6s
[CV] END ...................max_depth=None, n_estimators=170; total time=   0.5s
[CV] END ...................max_depth=None, n_estimators=170; total time=   0.6s
[CV] END ...................max_depth=None, n_estimators=170; total time=   0.5s
[CV] END ...................max_depth=None, n_estimators=170; total time=   0.5s
[CV] END ...................max_depth=None, n_estimators=200; total time=   0.6s
[CV] END ...................max_depth=None, n_estimators=200; total time=   0.6s
[CV] END ...................max_depth=None, n_estimators=200; total time=   0.6s
[CV] END ...................max_depth=None, n_estimators=200; total time=   0.6s
[CV] END ...................max_depth=None, n_estimators=200; total time=   0.7s
[CV] END ...................max_depth=None, n_estimators=230; total time=   0.7s
[CV] END ...................max_depth=None, n_estimators=230; total time=   0.7s
[CV] END ...................

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 3, 5, 7],
                         'n_estimators': [170, 200, 230]},
             scoring='recall', verbose=2)

In [17]:
grid.best_params_


{'max_depth': None, 'n_estimators': 200}

In [18]:
ypred = grid.best_estimator_.predict(Xtest)



In [19]:
ypred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [20]:
from sklearn.metrics import classification_report

print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.93      0.78      0.85      2003
           1       0.46      0.77      0.58       497

    accuracy                           0.78      2500
   macro avg       0.70      0.77      0.71      2500
weighted avg       0.84      0.78      0.79      2500

